In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import SimpleRNN, CuDNNLSTM, CuDNNGRU, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import Adam, SGD, RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [2]:
from keras.backend import manual_variable_initialization
manual_variable_initialization(True)

In [2]:
dataset = pd.read_csv('data/train.csv')
dataset.head()
dataset.shape

(20800, 5)

In [3]:
nan_indexes = dataset.loc[pd.isna(dataset["text"]), :].index
dataset = dataset.drop(nan_indexes)
dataset.shape

(20761, 5)

In [5]:
print(142 in nan_indexes)

True


In [6]:
titles = dataset.title

In [4]:
X = dataset.text
y = dataset.label

In [6]:
X[143]

'  U.N. Secretary General Complains That The ‘Masses Have Rejected Globalism’ In Favor Of Nationalism Antonio Guterres, elected in October to take over as U.N. secretary general next year, told a conference in his native Lisbon that this trend had undermined the willingness to receive refugees in Europe this year. He said the world must re-establish international protection for refugees coming from war zones such as Syria, but it would not be easy as developed countries were turning to nationalist agendas.   22, 2016 The incoming head of the United Nations warned on Tuesday that ‘losers of globalization’ in rich countries have felt ignored by establishment politicians, prompting them to turn to nationalist agendas, as in the U.S. election and Brexit referendum. “Therefore wait ye upon me, saith the LORD, until the day that I rise up to the prey: for my determination is to gather the nations, that I may assemble the kingdoms, to pour upon them mine indignation, even all my fierce anger:

In [8]:
mean_article_length = 0

for i in range(20761):
    if i in nan_indexes:
        continue
    mean_article_length += len(X[i].split(' '))

mean_article_length /= 20761

mean_article_length

773.6980395934685

In [5]:
max_words = 3000
max_len = 1000
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X)
sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [10]:
def LSTM_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,75,input_length=max_len)(inputs)
    layer = CuDNNLSTM(96)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [6]:
from keras.callbacks import ModelCheckpoint
import keras as K

def train_model(model_id, model, optimizer, epochs, X_train, y_train, validation_split, batch_size):

    model.summary()
    
    checkpoint = ModelCheckpoint('NNs/' + model_id + '-{epoch:03d}.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

    tb_callback = K.callbacks.TensorBoard(log_dir='./logs/nn_' + str(model_id))

    hist = model.fit(X_train, y_train, validation_split=validation_split, epochs=epochs, batch_size=batch_size, callbacks=[tb_callback, checkpoint])
    
    return hist

#hist = train_model(...)
#best_epoch = max(hist.history['val_loss'])



In [ ]:
model = LSTM_network()

train_model('LSTM 1 layer 75 words', model, Adam(), 30, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 1000, 75)          225000    
_________________________________________________________________
cu_dnnlstm_4 (CuDNNLSTM)     (None, 96)                66432     
_________________________________________________________________
out_layer (Dense)            (None, 1)                 97        
_________________________________________________________________
activation_5 (Activation)    (None, 1)                 0         
Total params: 291,529
Trainable params: 291,529
Non-trainable params: 0
_________________________________________________________________
Train on 16608 samples, validate on 4153 samples
Epoch 1/30
16608/16608 [==============================] - 8s 474us/step - loss: 0.4087 

In [ ]:
model.fit(sequences_matrix,y,batch_size=128,epochs=20,
          validation_split=0.2)


In [12]:
def RNN_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = SimpleRNN(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [16]:
model = RNN_network()

train_model('rnn', model, SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True), 30, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 1000, 50)          100000    
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 64)                7360      
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_7 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________

In [13]:
def GRU_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = GRU(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [14]:
model = GRU_network()

train_model('GRU', model, Adam(), 30, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 1000, 50)          100000    
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                22080     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_5 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________

In [20]:
def LSTM2_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(78)(layer)
    layer = Dense(128,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [21]:
model = LSTM2_network()

train_model('LSTM2', model, Adam(), 15, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 1000, 50)          100000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 78)                40248     
_________________________________________________________________
FC1 (Dense)                  (None, 128)               10112     
_________________________________________________________________
activation_13 (Activation)   (None, 128)               0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 129       
__________

In [22]:
max_words = 1500
max_len = 1000
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X)
sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [23]:
def LSTM3_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [24]:
model = LSTM3_network()

train_model('LSTM3', model, Adam(), 15, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 1000, 50)          75000     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_15 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________

In [26]:
def LSTM4_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,25,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [27]:
model = LSTM4_network()

train_model('LSTM4', model, Adam(), 15, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_9 (Embedding)      (None, 1000, 25)          37500     
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                23040     
_________________________________________________________________
out_layer (Dense)            (None, 1)                 65        
_________________________________________________________________
activation_17 (Activation)   (None, 1)                 0         
Total params: 60,605
Trainable params: 60,605
Non-trainable params: 0
_________________________________________________________________
Train on 16608 samples, validate on 4153 samples
Epoch 1/15
16608/16608 [==============================] - 53s 3ms/step - loss: 0.4125 - a

In [13]:
def LSTM5_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = CuDNNLSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [11]:
model = LSTM5_network()
train_model('LSTM5', model, Adam(), 15, sequences_matrix, y, 0.2, 128)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 1000, 50)          150000    
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 64)                29696     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
___________________________

In [16]:
def LSTM6_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words, 128, input_length=max_len)(inputs)
    layer = CuDNNLSTM(128)(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [17]:
model = LSTM6_network()
train_model('LSTM6', model, Adam(), 15, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 1000, 128)         384000    
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 128)               132096    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 129       
_________________________________________________________________
activation_3 (Activation)    (None, 1)                 0         
Total params: 516,225
Trainable params: 516,225
Non-trainable params: 0
_________________________________________________________________
Trai

In [19]:
def LSTM7_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words, 96, input_length=max_len)(inputs)
    layer = CuDNNLSTM(96)(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [20]:
model = LSTM7_network()
train_model('LSTM7', model, Adam(), 15, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 1000, 96)          288000    
_________________________________________________________________
cu_dnnlstm_3 (CuDNNLSTM)     (None, 96)                74496     
_________________________________________________________________
dropout_3 (Dropout)          (None, 96)                0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 97        
_________________________________________________________________
activation_4 (Activation)    (None, 1)                 0         
Total params: 362,593
Trainable params: 362,593
Non-trainable params: 0
_________________________________________________________________
Trai

In [23]:
def LSTM8_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words, 96, input_length=max_len)(inputs)
    layer = CuDNNLSTM(96)(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(24)(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [24]:
model = LSTM8_network()
train_model('LSTM8', model, Adam(), 15, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 1000, 96)          288000    
_________________________________________________________________
cu_dnnlstm_5 (CuDNNLSTM)     (None, 96)                74496     
_________________________________________________________________
dropout_6 (Dropout)          (None, 96)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 24)                2328      
_________________________________________________________________
dropout_7 (Dropout)          (None, 24)                0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 25        
__________

In [27]:
def LSTM9_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = CuDNNLSTM(64)(layer)
    layer = Dense(64,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [28]:
model = LSTM9_network()
train_model('LSTM9', model, Adam(), 15, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_9 (Embedding)      (None, 1000, 50)          150000    
_________________________________________________________________
cu_dnnlstm_6 (CuDNNLSTM)     (None, 64)                29696     
_________________________________________________________________
FC1 (Dense)                  (None, 64)                4160      
_________________________________________________________________
activation_7 (Activation)    (None, 64)                0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 65        
__________

In [29]:
def LSTM10_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = CuDNNLSTM(96)(layer)
    layer = Dense(96,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [30]:
model = LSTM10_network()
train_model('LSTM10', model, Adam(), 10, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_10 (Embedding)     (None, 1000, 50)          150000    
_________________________________________________________________
cu_dnnlstm_7 (CuDNNLSTM)     (None, 96)                56832     
_________________________________________________________________
FC1 (Dense)                  (None, 96)                9312      
_________________________________________________________________
activation_9 (Activation)    (None, 96)                0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 96)                0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 97        
__________

In [33]:
from keras import regularizers

In [31]:
def LSTM11_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = CuDNNLSTM(96)(layer)
    layer = Dense(96,name='FC1',kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [34]:
model = LSTM11_network()
train_model('LSTM11', model, Adam(), 10, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_12 (Embedding)     (None, 1000, 50)          150000    
_________________________________________________________________
cu_dnnlstm_9 (CuDNNLSTM)     (None, 96)                56832     
_________________________________________________________________
FC1 (Dense)                  (None, 96)                9312      
_________________________________________________________________
activation_11 (Activation)   (None, 96)                0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 96)                0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 97        
__________

KeyboardInterrupt: 

In [35]:
def LSTM12_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = CuDNNLSTM(64)(layer)
    layer = Dense(196,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [36]:
model = LSTM12_network()
train_model('LSTM12', model, Adam(), 10, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_13 (Embedding)     (None, 1000, 50)          150000    
_________________________________________________________________
cu_dnnlstm_10 (CuDNNLSTM)    (None, 64)                29696     
_________________________________________________________________
FC1 (Dense)                  (None, 196)               12740     
_________________________________________________________________
activation_13 (Activation)   (None, 196)               0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 196)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 197       
__________

In [37]:
def LSTM13_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = CuDNNLSTM(64)(layer)
    layer = Dense(196,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [39]:
model = LSTM13_network()
train_model('LSTM14', model, SGD(lr=0.1), 10, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_15 (Embedding)     (None, 1000, 50)          150000    
_________________________________________________________________
cu_dnnlstm_12 (CuDNNLSTM)    (None, 64)                29696     
_________________________________________________________________
FC1 (Dense)                  (None, 196)               12740     
_________________________________________________________________
activation_17 (Activation)   (None, 196)               0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 196)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 197       
__________

In [41]:
model = LSTM13_network()
train_model('LSTM15', model, RMSprop(), 10, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_16 (Embedding)     (None, 1000, 50)          150000    
_________________________________________________________________
cu_dnnlstm_13 (CuDNNLSTM)    (None, 64)                29696     
_________________________________________________________________
FC1 (Dense)                  (None, 196)               12740     
_________________________________________________________________
activation_19 (Activation)   (None, 196)               0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 196)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 197       
__________

In [43]:
def LSTM16_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = CuDNNLSTM(60)(layer)
    layer = Dense(196,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [44]:
model = LSTM16_network()
train_model('LSTM16', model, RMSprop(), 10, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_17 (Embedding)     (None, 1000, 50)          150000    
_________________________________________________________________
cu_dnnlstm_14 (CuDNNLSTM)    (None, 60)                26880     
_________________________________________________________________
FC1 (Dense)                  (None, 196)               11956     
_________________________________________________________________
activation_21 (Activation)   (None, 196)               0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 196)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 197       
__________

In [45]:
def LSTM17_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = CuDNNLSTM(68)(layer)
    layer = Dense(196,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [46]:
model = LSTM17_network()
train_model('LSTM17', model, RMSprop(), 10, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_18 (Embedding)     (None, 1000, 50)          150000    
_________________________________________________________________
cu_dnnlstm_15 (CuDNNLSTM)    (None, 68)                32640     
_________________________________________________________________
FC1 (Dense)                  (None, 196)               13524     
_________________________________________________________________
activation_23 (Activation)   (None, 196)               0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 196)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 197       
__________

In [47]:
def LSTM18_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = CuDNNLSTM(76)(layer)
    layer = Dense(196,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [48]:
model = LSTM18_network()
train_model('LSTM18', model, RMSprop(), 10, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_19 (Embedding)     (None, 1000, 50)          150000    
_________________________________________________________________
cu_dnnlstm_16 (CuDNNLSTM)    (None, 76)                38912     
_________________________________________________________________
FC1 (Dense)                  (None, 196)               15092     
_________________________________________________________________
activation_25 (Activation)   (None, 196)               0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 196)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 197       
__________

In [49]:
def LSTM19_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = CuDNNLSTM(76)(layer)
    layer = Dense(224,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [50]:
model = LSTM19_network()
train_model('LSTM19', model, RMSprop(), 10, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_20 (Embedding)     (None, 1000, 50)          150000    
_________________________________________________________________
cu_dnnlstm_17 (CuDNNLSTM)    (None, 76)                38912     
_________________________________________________________________
FC1 (Dense)                  (None, 224)               17248     
_________________________________________________________________
activation_27 (Activation)   (None, 224)               0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 224)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 225       
__________

In [52]:
def LSTM20_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = CuDNNLSTM(76)(layer)
    layer = Dense(296,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [53]:
model = LSTM20_network()
train_model('LSTM20', model, RMSprop(), 10, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_21 (Embedding)     (None, 1000, 50)          150000    
_________________________________________________________________
cu_dnnlstm_18 (CuDNNLSTM)    (None, 76)                38912     
_________________________________________________________________
FC1 (Dense)                  (None, 296)               22792     
_________________________________________________________________
activation_29 (Activation)   (None, 296)               0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 296)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 297       
__________

In [54]:
def LSTM21_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = CuDNNLSTM(76)(layer)
    layer = Dense(274,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [55]:
model = LSTM21_network()
train_model('LSTM21', model, RMSprop(), 10, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_22 (Embedding)     (None, 1000, 50)          150000    
_________________________________________________________________
cu_dnnlstm_19 (CuDNNLSTM)    (None, 76)                38912     
_________________________________________________________________
FC1 (Dense)                  (None, 274)               21098     
_________________________________________________________________
activation_31 (Activation)   (None, 274)               0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 274)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 275       
__________

In [58]:
def LSTM22_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = CuDNNLSTM(64)(layer)
    layer = Dense(196,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(64,name='FC2')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [60]:
model = LSTM22_network()
train_model('LSTM22', model, RMSprop(), 10, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_25 (Embedding)     (None, 1000, 50)          150000    
_________________________________________________________________
cu_dnnlstm_22 (CuDNNLSTM)    (None, 64)                29696     
_________________________________________________________________
FC1 (Dense)                  (None, 196)               12740     
_________________________________________________________________
activation_38 (Activation)   (None, 196)               0         
_________________________________________________________________
dropout_24 (Dropout)         (None, 196)               0         
_________________________________________________________________
FC2 (Dense)                  (None, 64)                12608     
__________

In [61]:
def LSTM23_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = CuDNNLSTM(64)(layer)
    layer = Dense(196,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(196,name='FC2')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [62]:
model = LSTM22_network()
train_model('LSTM22', model, RMSprop(), 10, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_26 (Embedding)     (None, 1000, 50)          150000    
_________________________________________________________________
cu_dnnlstm_23 (CuDNNLSTM)    (None, 64)                29696     
_________________________________________________________________
FC1 (Dense)                  (None, 196)               12740     
_________________________________________________________________
activation_41 (Activation)   (None, 196)               0         
_________________________________________________________________
dropout_26 (Dropout)         (None, 196)               0         
_________________________________________________________________
FC2 (Dense)                  (None, 64)                12608     
__________

In [10]:
def LSTM13_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = CuDNNLSTM(64)(layer)
    layer = Dense(196,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [11]:
model = LSTM13_network()
train_model('LSTM114', model, RMSprop(), 10, sequences_matrix, y, 0.2, 128)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 50)          150000    
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 64)                29696     
_________________________________________________________________
FC1 (Dense)                  (None, 196)               12740     
_________________________________________________________________
activation_1 (Activation)    (None, 196)               0         
_________________________________________________________________
dropout_1 (Dropout)  

FailedPreconditionError: Attempting to use uninitialized value FC1/bias
	 [[{{node FC1/bias/read}}]]
	 [[{{node loss/mul}}]]

Using TensorFlow backend.


In [7]:
def LSTMtest_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = CuDNNLSTM(64)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [8]:
model = LSTMtest_network()
train_model('LSTMtest', model, RMSprop(), 10, sequences_matrix, y, 0.2, 128)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 50)          150000    
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 64)                29696     
_________________________________________________________________
out_layer (Dense)            (None, 1)                 65        
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 179,761
Trainable params: 179,761
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Train on 16608 s

In [21]:
def LSTMtest2_network():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,64,input_length=max_len)(inputs)
    layer = CuDNNLSTM(48)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [22]:
model = LSTMtest2_network()
train_model('LSTMtest2', model, RMSprop(), 10, sequences_matrix, y, 0.2, 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 1000)              0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 1000, 64)          192000    
_________________________________________________________________
cu_dnnlstm_3 (CuDNNLSTM)     (None, 48)                21888     
_________________________________________________________________
out_layer (Dense)            (None, 1)                 49        
_________________________________________________________________
activation_5 (Activation)    (None, 1)                 0         
Total params: 213,937
Trainable params: 213,937
Non-trainable params: 0
_________________________________________________________________
Train on 16608 samples, validate on 4153 samples
Epoch 1/10
16608/16608 [==============================] - 8s 470us/step - loss: 0.3877 